In [ ]:
IS_COLAB = False
if 'google.colab' in str(get_ipython()):
  IS_COLAB = True

In [ ]:
!nvidia-smi

## Section 0

**What's this now?**

"Generative Pre-trained Transformer" (GPT) models are a variety of neural network that attempts to complete a block of text according to a pretrained concept of how text should be completed. For example, a model trained by viewing numerous chat logs would, during inference/generation, attempt to continue a partial conversation that is fed to it as a chat log. Another talent shown by these models is summarization, e.g. if a large block of text is followed by "tl;dr:" a model trained on many online posts will know to generate a "Too long; didn't read:" summary of the preceding text.

There are many classes of GPT-type models, but the biggest one available (for local inference) as of this notebook's creation is GPT-NeoX-20B. However, training it requires compute that most people don't have access to. And even running inference is a challenge as the full weights are larger than any consumer grade GPU's vRAM.

**How the?**

If we're just talking about abstract models somewhere out there, FANG+M all appear to be running one or another version of the 185B-param GPT3 (19x bigger than NeoX-20B). You can poke around with OpenAI's davinci, but it's not downloadable, and inference requires a small team of datacenter techs.

In 2021 a loose network of eccentric personalities and interested parties known as EleutherAI generously released [a free pretrained weights file](https://blog.eleuther.ai/announcing-20b/), which is currently [available for download at huggingface's repo](https://huggingface.co/EleutherAI/gpt-neox-20b). To create this file the EleutherAI chaps repeatedly showed the network samples from an ~800GB text dataset known as "The Pile", which contains whatever symbol arrangements the EleutherAI people think is pertinent to the bettering of humanity.

This notebook can run "locally" on a colab pro instance, or actually locally on two rtx3090s with nvlink and >90GB of CPU RAM. It should be easy to modify for bigger setups and harder for smaller ones. There are numerous online tutorials explaining that inference on NeoX-20B **runs great on two 3090s with just one simple trick**. Maybe the bottleneck is just in the sparseness of explored hardware configurations, but there are a lot of people raising issues with those setups. In this case I used facebook's bitsandbytes and huggingface's accelerate modules to shrink the model weights and distribute the inference load according to one's GPU setup.

Reducing the model size (and in the dual 3090 setup, splitting things between gpus) was enough with the hardware described. For those with colab non-pro accounts or otherwise smaller setups, you may have luck using [Microsoft's deepspeed](https://github.com/microsoft/DeepSpeed) or one of [the other huggingface tools](https://huggingface.co/docs/transformers/big_models). However you should realistically expect to see a huge increase in inference time. Just because you can technically run this on an orangePi and 10TB of HDD swap doesn't mean it will finish before you retire.

**Who the?**

Notebook Author: https://github.com/calhounpaul

## Section 1: Prereqs

If docker's your thing, here's what I've been running this in:
```
nvidia-docker run -it \
           --gpus all \
           -p 8888:8888 \
           -v $HOME/neox_docker:/root/neox_docker \
           -v $HOME/.cache:/root/.cache \
           -w /root/neox_docker \
           huggingface/transformers-deepspeed:latest-training
```

But my deepspeed container here is overkill. You'd probably do fine with [huggingface/transformers-pytorch-gpu:latest](https://hub.docker.com/r/huggingface/transformers-pytorch-gpu/tags).

**If you're in colab, you don't need docker (but you do still need a pro instance, for now).**

In [ ]:
!pip install accelerate
!pip install --upgrade transformers
!pip install bitsandbytes
#!pip install truecase unidecode ftfy #useful tools for handling text
#!pip install gradio

## Section 2: Loading the Model

In [ ]:
from transformers import logging
logging.set_verbosity(logging.ERROR)

In [ ]:
import torch
from accelerate import init_empty_weights, infer_auto_device_map, load_checkpoint_and_dispatch
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, logging
import os
import argparse
torch.cuda.empty_cache()

This example loads "EleutherAI/gpt-neox-20b" from HF.

Other options include:
*   FrostAura/gpt-neox-20b-fiction-novel-generation
*   larrylawl/gpt-neox-20b-shakespeare
*   KoboldAI/GPT-NeoX-20B-Skein

The first two were fine tuned from the original EleutherAI model. This means they probably don't have much new data in them, but rather their output is more constrained to one or more specific purposes. But the KoboldAI model appears to be trained from scratch.

In [ ]:
model_name = "EleutherAI/gpt-neox-20b"

I've made extensive use of the HF "Auto___" classes for configs and the like in the hopes that this notebook will continue to function for more than a week.

In [ ]:
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

The first trick is to use accelerate's "init_empty_weights" to figure out how to load things into the provided setup.

In [ ]:
with init_empty_weights():
    model = AutoModelForCausalLM.from_config(config)

Then we can specify what needs to go where, what can/can't be optimized, and what datatype we'll eventually use. This results in an efficient device map that we can plug into the pretrained model when it's instantiated next.

In [ ]:
device_map = infer_auto_device_map(model,
                                  max_memory={0: "34GiB", "cpu": "80GiB"} if IS_COLAB else {0: "19GiB",1: "19GiB", "cpu": "90GiB"},
                                  no_split_module_classes=["GPTNeoXLayer"],
                                  dtype=torch.int8
                              )

And now to put it all together **(warning: this bit takes about 10 minutes at HF-to-colab's respectable 85MB/s)**:


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
                                  model_name,
                                  device_map=device_map,
                                  config=config,
                                  load_in_8bit=True
                              )

## Section 3: Messing Around

Now to do some few-shot inference.

Here's a template for a simple Q&A:
```
<context>About 1850 Lincoln compiled notes for an address to young men on the subject of the profession of law. Here
again we find a refreshingly candid approach, looking without pretense at the creature man. One piece of advice
which Lincoln urged upon young lawyers was that they never take their whole fee in advance. To do so would place too
great a strain upon human nature, which would then lack the needful spur to industry. “When fully paid beforehand, you
are more than a common mortal if you can feel the same interest in the case, as if something was still in prospect
for you, as well as for your client.” As in the case of the subtreasury bill, Lincoln saw the yoking of duty and
self-interest as a necessity of our nature.</context>

<begin chat>

Question: What is the capital of France?
Answer: The capital of France is Paris.
Question: How many states are in the United States?
Answer: There are 50 states in the United States.
Question: What is the population of Canada?
Answer: The population of Canada is 37.59 million people as of July 1, 2020.
Question: What were Lincoln's beliefs on accepting prepayment for professional services?
Answer:
```
What this seems to do is force the model to text complete in a way that answers the question, given the preceding context. None of the tag formats were intentionally added for training. It figures them out from the context and then follows along.

In [ ]:
query_in = "<context>About 1850 Lincoln compiled notes for an address to young men on the subject of the profession of law. Here again we find a refreshingly candid approach, looking without pretense at the creature man. One piece of advice which Lincoln urged upon young lawyers was that they never take their whole fee in advance. To do so would place too great a strain upon human nature, which would then lack the needful spur to industry. “When fully paid beforehand, you are more than a common mortal if you can feel the same interest in the case, as if something was still in prospect for you, as well as for your client.” As in the case of the subtreasury bill, Lincoln saw the yoking of duty and self-interest as a necessity of our nature.</context>\n\n<begin chat>\n\nQuestion: What is the capital of France?\nAnswer: The capital of France is Paris.\nQuestion: How many states are in the United States?\nAnswer: There are 50 states in the United States.\nQuestion: What is the population of Canada?\nAnswer: The population of Canada is 37.59 million people as of July 1, 2020.\nQuestion: What were Lincoln's beliefs on accepting prepayment for professional services?\nAnswer:"

In [ ]:
input_ids = tokenizer(query_in, return_tensors="pt").input_ids
input_token_length=input_ids.shape[1]
input_ids = input_ids.to('cuda')
output_tokens = model.generate(
        input_ids,
        do_sample=True,
        temperature=0.9,
        max_length=input_token_length+256,
    )
output_text = tokenizer.batch_decode(output_tokens)[0]
print(output_text)

I have no idea what was generated by your use of the above code, but if it's something terrible please send all complaints to the EleutherAI folks, or maybe to HuggingFace, or you can put the model on an SD card and yell at that.

Some of the raw "Answer:" completions I've generated in the past few minutes are:

* *Lincoln recommended that lawyers don’t accept bills in advance as payment for legal work.*

*  *Lincoln stated that accepting prepayment for professional services was a poor business practice.*

*  *Lincoln believed that taking “all your fee in advance” would place too much of a strain on human nature, and result in “not the same interest in the case.”*

*  *Lincoln advised young lawyers to never take a full fee in advance.*

*  *Lincolns advice to young lawyers to not take their fee in advance, to avoid the strain of taking a fee and the lack of needful incentive to the lawyer to work diligently for the clients cause.*

You'll notice much of the output you get is unnecessary (the above examples are truncated after the answer).

The following class allows you to add custom halt tokens. However, you may want to take a look at [how tokenization works under the hood](https://huggingface.co/docs/transformers/main_classes/tokenizer) before you start adding special cases.

A simpler technique to get e.g. a chatbot to halt after generating one response is to add one or two "<|endoftext|>" tokens after each bot response in the preprompt. This should usually few-shot "train" the model that it should stop after one reply (as opposed to generating the human response).

In [ ]:
from transformers import StoppingCriteria
#modified from the EndOfFunctionCriteria class I found somewhere around here: https://huggingface.co/transformers/v4.6.0/_modules/transformers/generation_stopping_criteria.html
class EndOfQuestionCriteria(StoppingCriteria):
  def __init__(self, start_length, eof_strings, tokenizer):
    self.start_length = start_length
    self.eof_strings = eof_strings
    self.tokenizer = tokenizer
  def __call__(self, input_ids, scores, **kwargs):
    decoded_generations = self.tokenizer.batch_decode(input_ids[:, self.start_length :])
    done = []
    for decoded_generation in decoded_generations:
      done.append(any([stop_string in decoded_generation for stop_string in self.eof_strings]))
    return all(done)

Then add the criteria when calling model.generate()

In [ ]:
from transformers import StoppingCriteriaList

extra_eos = ["Human:","AI:","Question:","Answer:","</"]

input_ids = tokenizer(query_in, return_tensors="pt").input_ids
input_token_length=input_ids.shape[1]
input_ids = input_ids.to('cuda')
output_tokens = model.generate(
    input_ids,
    stopping_criteria=StoppingCriteriaList([EndOfQuestionCriteria(
            input_token_length,
            extra_eos,
            tokenizer,
        ),]),
    do_sample=True,
    temperature=0.9,
    max_length=input_token_length+256,
)
output_text = tokenizer.batch_decode(output_tokens)[0]

for eos_str in extra_eos:
  if output_text.endswith(eos_str):
    output_text = output_text[:-len(eos_str)].rstrip()
    break

print(output_text)

### TBC